### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 1

today = date.today()
today

datetime.date(2022, 5, 10)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
1,ADVANC,8085,10,26589096,4
11,AOT,8085,10,-17151697,4
12,AP,8085,10,4870212,4
13,ASIAN,8085,10,1077825,4
14,ASK,8085,10,1277644,4
15,ASP,8085,10,775000,4
20,BANPU,8085,10,18580896,4
21,BAY,8085,10,34707618,4
22,BBL,8085,10,26702007,4
25,BCPG,8085,10,2850502,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 1-1) 
OR (year = 2022-1 AND quarter >= 1) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(217, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ADVANC,2022,Q1,26589096,26922146,-333050,-1.24
1,AOT,2022,Q1,-17151697,-16322014,-829683,-5.08
2,AP,2022,Q1,4870212,4543060,327152,7.20
3,ASIAN,2022,Q1,1077825,1044316,33509,3.21
4,ASK,2022,Q1,1277644,1202804,74840,6.22
5,ASP,2022,Q1,775000,978355,-203355,-20.79
6,BANPU,2022,Q1,18580896,9851795,8729101,88.60
7,BAY,2022,Q1,34707618,33794188,913430,2.70
8,BBL,2022,Q1,26702007,26507039,194968,0.74
9,BCPG,2022,Q1,2850502,2010821,839681,41.76


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q1'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

49

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(403, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

70

In [11]:
for rcd in rcds:
    print(rcd)

['ADVANC', 2022, 'Q1', 26589096, 26922146, -333050, -1.24, 6]
['AOT', 2022, 'Q1', -17151697, -16322014, -829683, -5.08, 24]
['AP', 2022, 'Q1', 4870212, 4543060, 327152, 7.2, 25]
['ASIAN', 2022, 'Q1', 1077825, 1044316, 33509, 3.21, 36]
['ASK', 2022, 'Q1', 1277644, 1202804, 74840, 6.22, 38]
['ASP', 2022, 'Q1', 775000, 978355, -203355, -20.79, 40]
['BANPU', 2022, 'Q1', 18580896, 9851795, 8729101, 88.6, 47]
['BAY', 2022, 'Q1', 34707618, 33794188, 913430, 2.7, 49]
['BBL', 2022, 'Q1', 26702007, 26507039, 194968, 0.74, 50]
['BCPG', 2022, 'Q1', 2850502, 2010821, 839681, 41.76, 53]
['BFIT', 2022, 'Q1', 696050, 906775, -210725, -23.24, 58]
['BH', 2022, 'Q1', 1849587, 1215678, 633909, 52.14, 61]
['BJC', 2022, 'Q1', 3818410, 3584811, 233599, 6.52, 65]
['BPP', 2022, 'Q1', 5011201, 3127027, 1884174, 60.25, 74]
['COTTO', 2022, 'Q1', 608126, 583604, 24522, 4.2, 710]
['CPNREIT', 2022, 'Q1', 869788, 865890, 3898, 0.45, 647]
['DCC', 2022, 'Q1', 1741605, 1700429, 41176, 2.42, 135]
['DCON', 2022, 'Q1', 119

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
21,GGC,2022,Q1,722960,330219,392741,118.93
6,BANPU,2022,Q1,18580896,9851795,8729101,88.60
13,BPP,2022,Q1,5011201,3127027,1884174,60.25
60,THG,2022,Q1,2079171,1337425,741746,55.46
11,BH,2022,Q1,1849587,1215678,633909,52.14


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
6,BANPU,2022,Q1,18580896,9851795,8729101,88.60
13,BPP,2022,Q1,5011201,3127027,1884174,60.25
60,THG,2022,Q1,2079171,1337425,741746,55.46
11,BH,2022,Q1,1849587,1215678,633909,52.14
9,BCPG,2022,Q1,2850502,2010821,839681,41.76


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
21,GGC,2022,Q1,722960,330219,392741,118.93
6,BANPU,2022,Q1,18580896,9851795,8729101,88.60
13,BPP,2022,Q1,5011201,3127027,1884174,60.25
60,THG,2022,Q1,2079171,1337425,741746,55.46
11,BH,2022,Q1,1849587,1215678,633909,52.14


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
6,BANPU,2022,Q1,18580896,9851795,8729101,88.60
13,BPP,2022,Q1,5011201,3127027,1884174,60.25
60,THG,2022,Q1,2079171,1337425,741746,55.46
11,BH,2022,Q1,1849587,1215678,633909,52.14
9,BCPG,2022,Q1,2850502,2010821,839681,41.76
